<a href="https://colab.research.google.com/github/ngtridung97/Wikipedia-Playground/blob/main/wiki_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
from bs4 import BeautifulSoup

In [ ]:
target_word = 'Ending'
start_url = 'https://en.wikipedia.org/wiki/Special:Random'
target_url = 'https://en.wikipedia.org/wiki/' + target_word

In [ ]:
# Lmit loop number
def continue_crawl(search_history, target_url, max_links=30):
    if search_history[-1] == target_url:													
        print('Article ' + target_word + ' is found.')
        return False
    elif len(search_history) > max_links:
        print('Article ' + target_word + ' is aborted at ' + str(max_links) + ' search.')
        return False
    elif len(search_history) != len(set(search_history)):
        search_word = search_history[-1][30:]
        print('Article ' + search_word + ' is overlapped.')
        return False
    else:
        return True

# Define next url by paragraph
def next_url(url, paragraph_no=1, word_no=1):
	response = requests.get(url)
	html = response.text
	soup = BeautifulSoup(html, 'html.parser')
	
	content_div = soup.find(id='mw-content-text').find(class_='mw-parser-output')

	article_link = None
	i = 0
	for paragraph in content_div.find_all('p', recursive=False):
		if paragraph.find(class_=lambda x: x not in ['mw-empty-elt']):
			word = paragraph.find_all('a', recursive=False)
			i += 1
			if i == paragraph_no and len(word) >= word_no:
				article_link = word[word_no - 1].get('href')
				break

	if not article_link:
		return
	
	next_link = 'https://en.wikipedia.org' + article_link
	return next_link

In [ ]:
article_chain = [start_url]

print('Start url: ' + start_url)
print('Target url: ' + target_url + '\n')

i = 0
while continue_crawl(article_chain, target_url):
	print('Looping ' + str(i).zfill(2) + ': ' + article_chain[-1])
	next_link = next_url(article_chain[-1])
	i += 1

	if not next_link:
		print('Article ' + target_word +  ' is aborted, no next link found.')
		break
	
	article_chain.append(next_link)
	time.sleep(0.25)

print('\nDone!')